# dataset


GTCrossView images [here](https://www.mediafire.com/folder/f4gga3h86d659/GTCrossView)


Refine_Net segmentation map [here](https://drive.google.com/file/d/1DXSSMOAUC5g8813k938v58oY-GvdSjJL/view)



In [ ]:
import numpy as np
import cv2 as cv
import glob
import os

I will split the dayton images into train and test subsets according to the **AsBs** folder.

pipeline:

```
read all images in train or test folder -> 
read the corresponding aerial and street images in the dayton folder -> 
concate them together -> 
put the resulting concat image into the new train or test folder

```

In [ ]:
data_root = '/media/zikui/Data/xview_data'
asbs_train_paths = os.path.join(data_root,'AsBs/train')
asbs_test_paths = os.path.join(data_root,'AsBs/test')
dayton_path = os.path.join(data_root,'dayton')

asbs_train = glob.glob(os.path.join(asbs_train_paths, "*.png"))
asbs_test = glob.glob(os.path.join(asbs_test_paths, "*.png"))
dayton = glob.glob(os.path.join(dayton_path, "*.jpg"))

asbs_train = sorted(asbs_train, key=lambda x: x.split('/')[-1])
asbs_test = sorted(asbs_test, key=lambda x: x.split('/')[-1])
dayton = sorted(dayton, key=lambda x: x.split('/')[-1])

# sanity check 
print('dataset same size?',len(asbs_train) + len(asbs_test) == len(dayton)/2)

In [ ]:
# make train and test direcotries
ABAsBs = os.path.join(data_root,'ABAsBs')
ABAsBs_train = os.path.join(data_root,'ABAsBs/train')
ABAsBs_test = os.path.join(data_root,'ABAsBs/test')
os.makedirs(ABAsBs, exist_ok=False)
os.makedirs(ABAsBs_train, exist_ok=False)
os.makedirs(ABAsBs_test, exist_ok=False)
train_index = 0
test_index = 0
# since the test dataset is too big, we only include 200 images from it.
test_maxsize = 200

# for i in range(100):
for i in range(len(dayton)):
    img_name = dayton[i].split('/')[-1]
    if img_name.split('.')[-2] == 'overhead':
        img_id = img_name[:-13]
        print('img_id:',img_id)
        img_A_path = dayton[i]
        img_G_path = dayton[i+1]
        img_A = cv.imread(img_A_path)
        img_G = cv.imread(img_G_path)
        size = (256,256)
        img_A = cv.resize(img_A, size, interpolation = cv.INTER_AREA)
        img_G = cv.resize(img_G, size, interpolation = cv.INTER_AREA)
        img_GA = np.concatenate((img_G, img_A), axis=1)
        
        seg_train_name = asbs_train[0].split('/')[-1]
        seg_train_id = seg_train_name[:-4]
        seg_test_name = asbs_test[0].split('/')[-1]
        seg_test_id = seg_test_name[:-4]
        
        if img_id == seg_train_id:
            train_index += 1
            seg_train = cv.imread(asbs_train[0])
            img_GAs = np.concatenate((img_GA, seg_train), axis=1)
            img_GAs_path = os.path.join(ABAsBs_train,str(train_index)+'.jpg')
            cv.imwrite(img_GAs_path,img_GAs)
            asbs_train.pop(0)
            
        elif test_index < test_maxsize and img_id == seg_test_id:
            test_index += 1
            seg_test = cv.imread(asbs_test[0])
            img_GAs = np.concatenate((img_GA, seg_test), axis=1)
            img_GAs_path = os.path.join(ABAsBs_test,str(test_index)+'.jpg')
            cv.imwrite(img_GAs_path,img_GAs)
            asbs_test.pop(0)
            
        else: raise NameError('img_id not train nor test!')
#         overhead_name = dayton[i].split('/')[-1][:-13]
#         print(overhead_name)
#     elif img_name.split('.')[-2] == 'streetview':
#         streetview_name = dayton[i].split('/')[-1][:-15]
#         print(streetview_name)
#     else: print('error')